In [1]:
# install required libraries
!pip3 install sentence-transformers

     |████████████████████████████████| 71kB 5.2MB/s 
     |████████████████████████████████| 1.9MB 7.3MB/s 
     |████████████████████████████████| 1.2MB 37.5MB/s 
     |████████████████████████████████| 890kB 53.8MB/s 
     |████████████████████████████████| 3.2MB 47.1MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.4.1.2-cp37-none-any.whl size=103068 sha256=71903f02baf39aa33a537f896ba244211a98702d97c52b74e962276b3542950f
  Stored in directory: /root/.cache/pip/wheels/3d/33/d1/5703dd56199c09d4a1b41e0c07fb4e7765a84d787cbdc48ac3
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=f8b95816b1fc76937e202a182ec8f1f8dc2ad4b443e63c094aad1167d9774cef
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sentence-transformers sacremoses


In [2]:
!pip3 install elasticsearch

     |████████████████████████████████| 327kB 7.8MB/s 


In [3]:
from google.colab import drive

In [4]:
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
%cd /content/drive/MyDrive/BERT-FAQ

/content/drive/MyDrive/BERT-FAQ


In [6]:
!ls

data			     metric.py	  reranker.py
evaluation.py		     notebook	  searcher.py
faq_bert_finetuning.py	     output	  shared
faq_bert.py		     parser	  training_data_generator.py
hard_negatives_generator.py  __pycache__
indexer.py		     README.md


In [7]:
# import required dependencies
from evaluation import get_relevance_label_df
from shared.utils import load_from_json
from shared.utils import dump_to_json
from shared.utils import make_dirs
from reranker import ReRanker

**#################### Triplet Loss ######################**

In [8]:
output_path="data/FAQIR/rank_results"

# load user_query ES results from json files
es_output_path = output_path + "/unsupervised"
es_query_by_question = load_from_json(es_output_path + '/es_query_by_question.json')
es_query_by_answer = load_from_json(es_output_path + '/es_query_by_answer.json')
es_query_by_question_answer = load_from_json(es_output_path + '/es_query_by_question_answer.json')
es_query_by_question_answer_concat = load_from_json(es_output_path + '/es_query_by_question_answer_concat.json')

In [9]:
# load test_queries, relevance_label_df for ReRanker
query_answer_pair_filepath = 'data/FAQIR/query_answer_pairs.json'
relevance_label_df = get_relevance_label_df(query_answer_pair_filepath)
test_queries = relevance_label_df[relevance_label_df['query_type'] == 'user_query'].question.unique()

In [10]:
test_queries[:10]

array(['How to make font strikethrough on github.',
       'Is it possible to get  strikethrough letter formatting on github markdown.',
       'Making the text on github crossed out.',
       'Introducing stikethrough formatting on markdown for github.',
       'The <s> tag for font on github markdown doesnt work, is there an alternative?',
       'Making the letters i write on github striked through.',
       'Producing strikethrough text in github.',
       'Does github support strikethrough letters?',
       'How can I cross out my text on git hub?',
       'I want to have strikethrough text on github, is this possible?'],
      dtype=object)

In [11]:
# total number of test queries
len(test_queries)

1249

**Re-ranking results using query_type="user_query"; neg_type="hard"**

In [12]:
# define variables
query_type="user_query"; neg_type="hard"; version="1.1"; loss_type='triplet'
bert_model_path='output/FAQIR/models/' + loss_type + '_' + neg_type + '_' + query_type + '_' + version

In [13]:
# create instance of ReRanker class
r = ReRanker(
    bert_model_path=bert_model_path, 
    test_queries=test_queries, relevance_label_df=relevance_label_df, loss_type=loss_type
)

In [14]:
# next, generate BERT top-k preds and save to files
bert_query_by_question = r.get_bert_topk_preds(es_query_by_question)
bert_query_by_answer = r.get_bert_topk_preds(es_query_by_answer)
bert_query_by_question_answer = r.get_bert_topk_preds(es_query_by_question_answer)
bert_query_by_question_answer_concat = r.get_bert_topk_preds(es_query_by_question_answer_concat)

2021-02-28 08:30:56 - Generating BERT top-k results ...
2021-02-28 08:30:56 - Load pretrained SentenceTransformer: output/StackFAQv1.0/models/triplet_hard_user_query_1.1
2021-02-28 08:30:57 - Load SentenceTransformer from folder: output/StackFAQv1.0/models/triplet_hard_user_query_1.1
2021-02-28 08:31:15 - Use pytorch device: cuda
100%|██████████| 1249/1249 [01:56<00:00, 10.71it/s]
2021-02-28 08:33:12 - Generating BERT top-k results ...
2021-02-28 08:33:12 - Load pretrained SentenceTransformer: output/StackFAQv1.0/models/triplet_hard_user_query_1.1
2021-02-28 08:33:12 - Load SentenceTransformer from folder: output/StackFAQv1.0/models/triplet_hard_user_query_1.1
2021-02-28 08:33:13 - Use pytorch device: cuda
100%|██████████| 1249/1249 [01:49<00:00, 11.41it/s]
2021-02-28 08:35:03 - Generating BERT top-k results ...
2021-02-28 08:35:03 - Load pretrained SentenceTransformer: output/StackFAQv1.0/models/triplet_hard_user_query_1.1
2021-02-28 08:35:03 - Load SentenceTransformer from folder: ou

In [15]:
# dump BERT top-k preds to json files
bert_output_path = output_path + "/supervised/" + loss_type + "/" + query_type + "/" + neg_type
make_dirs(bert_output_path)

dump_to_json(bert_query_by_question, bert_output_path + '/bert_query_by_question.json')
dump_to_json(bert_query_by_answer, bert_output_path + '/bert_query_by_answer.json')
dump_to_json(bert_query_by_question_answer, bert_output_path + '/bert_query_by_question_answer.json')
dump_to_json(bert_query_by_question_answer_concat, bert_output_path + '/bert_query_by_question_answer_concat.json')

In [16]:
# next, generate the Re-ranked top-k results 
reranked_query_by_question = r.get_reranked_results(bert_query_by_question)
reranked_query_by_answer = r.get_reranked_results(bert_query_by_answer)
reranked_query_by_question_answer = r.get_reranked_results(bert_query_by_question_answer)
reranked_query_by_question_answer_concat = r.get_reranked_results(bert_query_by_question_answer_concat)

2021-02-28 08:38:40 - Re-ranking the top-k results ...
2021-02-28 08:38:40 - Re-ranking the top-k results ...
2021-02-28 08:38:40 - Re-ranking the top-k results ...
2021-02-28 08:38:40 - Re-ranking the top-k results ...


In [17]:
# dump Re-ranked results to json files
reranked_output_path = output_path + "/supervised/" + loss_type + "/" + query_type + "/" + neg_type
make_dirs(reranked_output_path)

dump_to_json(reranked_query_by_question, reranked_output_path + '/reranked_query_by_question.json')
dump_to_json(reranked_query_by_answer, reranked_output_path + '/reranked_query_by_answer.json')
dump_to_json(reranked_query_by_question_answer, reranked_output_path + '/reranked_query_by_question_answer.json')
dump_to_json(reranked_query_by_question_answer_concat, reranked_output_path + '/reranked_query_by_question_answer_concat.json')

**Re-ranking results using query_type="user_query"; neg_type="simple"**

In [18]:
# define variables
query_type="user_query"; neg_type="simple"; version="1.1"; loss_type='triplet'
bert_model_path='output/FAQIR/models/' + loss_type + '_' + neg_type + '_' + query_type + '_' + version

In [19]:
# create instance of ReRanker class
r = ReRanker(
    bert_model_path=bert_model_path, 
    test_queries=test_queries, relevance_label_df=relevance_label_df, loss_type=loss_type
)

In [20]:
# next, generate BERT top-k preds and save to files
bert_query_by_question = r.get_bert_topk_preds(es_query_by_question)
bert_query_by_answer = r.get_bert_topk_preds(es_query_by_answer)
bert_query_by_question_answer = r.get_bert_topk_preds(es_query_by_question_answer)
bert_query_by_question_answer_concat = r.get_bert_topk_preds(es_query_by_question_answer_concat)

2021-02-28 08:38:40 - Generating BERT top-k results ...
2021-02-28 08:38:40 - Load pretrained SentenceTransformer: output/StackFAQv1.0/models/triplet_simple_user_query_1.1
2021-02-28 08:38:40 - Load SentenceTransformer from folder: output/StackFAQv1.0/models/triplet_simple_user_query_1.1
2021-02-28 08:38:57 - Use pytorch device: cuda
100%|██████████| 1249/1249 [01:45<00:00, 11.88it/s]
2021-02-28 08:40:42 - Generating BERT top-k results ...
2021-02-28 08:40:42 - Load pretrained SentenceTransformer: output/StackFAQv1.0/models/triplet_simple_user_query_1.1
2021-02-28 08:40:42 - Load SentenceTransformer from folder: output/StackFAQv1.0/models/triplet_simple_user_query_1.1
2021-02-28 08:40:43 - Use pytorch device: cuda
100%|██████████| 1249/1249 [01:47<00:00, 11.64it/s]
2021-02-28 08:42:31 - Generating BERT top-k results ...
2021-02-28 08:42:31 - Load pretrained SentenceTransformer: output/StackFAQv1.0/models/triplet_simple_user_query_1.1
2021-02-28 08:42:31 - Load SentenceTransformer from 

In [21]:
# dump BERT top-k preds to json files
bert_output_path = output_path + "/supervised/" + loss_type + "/" + query_type + "/" + neg_type
make_dirs(bert_output_path)

dump_to_json(bert_query_by_question, bert_output_path + '/bert_query_by_question.json')
dump_to_json(bert_query_by_answer, bert_output_path + '/bert_query_by_answer.json')
dump_to_json(bert_query_by_question_answer, bert_output_path + '/bert_query_by_question_answer.json')
dump_to_json(bert_query_by_question_answer_concat, bert_output_path + '/bert_query_by_question_answer_concat.json')

In [22]:
# next, generate the Re-ranked top-k results 
reranked_query_by_question = r.get_reranked_results(bert_query_by_question)
reranked_query_by_answer = r.get_reranked_results(bert_query_by_answer)
reranked_query_by_question_answer = r.get_reranked_results(bert_query_by_question_answer)
reranked_query_by_question_answer_concat = r.get_reranked_results(bert_query_by_question_answer_concat)

2021-02-28 08:46:05 - Re-ranking the top-k results ...
2021-02-28 08:46:06 - Re-ranking the top-k results ...
2021-02-28 08:46:06 - Re-ranking the top-k results ...
2021-02-28 08:46:06 - Re-ranking the top-k results ...


In [23]:
# dump Re-ranked results to json files
reranked_output_path = output_path + "/supervised/" + loss_type + "/" + query_type + "/" + neg_type
make_dirs(reranked_output_path)

dump_to_json(reranked_query_by_question, reranked_output_path + '/reranked_query_by_question.json')
dump_to_json(reranked_query_by_answer, reranked_output_path + '/reranked_query_by_answer.json')
dump_to_json(reranked_query_by_question_answer, reranked_output_path + '/reranked_query_by_question_answer.json')
dump_to_json(reranked_query_by_question_answer_concat, reranked_output_path + '/reranked_query_by_question_answer_concat.json')

**Re-ranking results using query_type="faq"; neg_type="hard"**

In [24]:
# define variables
query_type="faq"; neg_type="hard"; version="1.1"; loss_type='triplet'
bert_model_path='output/FAQIR/models/' + loss_type + '_' + neg_type + '_' + query_type + '_' + version

In [25]:
# create instance of ReRanker class
r = ReRanker(
    bert_model_path=bert_model_path, 
    test_queries=test_queries, relevance_label_df=relevance_label_df, loss_type=loss_type
)

In [26]:
# next, generate BERT top-k preds and save to files
bert_query_by_question = r.get_bert_topk_preds(es_query_by_question)
bert_query_by_answer = r.get_bert_topk_preds(es_query_by_answer)
bert_query_by_question_answer = r.get_bert_topk_preds(es_query_by_question_answer)
bert_query_by_question_answer_concat = r.get_bert_topk_preds(es_query_by_question_answer_concat)

2021-02-28 08:46:07 - Generating BERT top-k results ...
2021-02-28 08:46:07 - Load pretrained SentenceTransformer: output/StackFAQv1.0/models/triplet_hard_faq_1.1
2021-02-28 08:46:07 - Load SentenceTransformer from folder: output/StackFAQv1.0/models/triplet_hard_faq_1.1
2021-02-28 08:46:25 - Use pytorch device: cuda
100%|██████████| 1249/1249 [01:43<00:00, 12.05it/s]
2021-02-28 08:48:09 - Generating BERT top-k results ...
2021-02-28 08:48:09 - Load pretrained SentenceTransformer: output/StackFAQv1.0/models/triplet_hard_faq_1.1
2021-02-28 08:48:09 - Load SentenceTransformer from folder: output/StackFAQv1.0/models/triplet_hard_faq_1.1
2021-02-28 08:48:10 - Use pytorch device: cuda
100%|██████████| 1249/1249 [01:46<00:00, 11.68it/s]
2021-02-28 08:49:57 - Generating BERT top-k results ...
2021-02-28 08:49:57 - Load pretrained SentenceTransformer: output/StackFAQv1.0/models/triplet_hard_faq_1.1
2021-02-28 08:49:57 - Load SentenceTransformer from folder: output/StackFAQv1.0/models/triplet_ha

In [27]:
# dump BERT top-k preds to json files
bert_output_path = output_path + "/supervised/" + loss_type + "/" + query_type + "/" + neg_type
make_dirs(bert_output_path)

dump_to_json(bert_query_by_question, bert_output_path + '/bert_query_by_question.json')
dump_to_json(bert_query_by_answer, bert_output_path + '/bert_query_by_answer.json')
dump_to_json(bert_query_by_question_answer, bert_output_path + '/bert_query_by_question_answer.json')
dump_to_json(bert_query_by_question_answer_concat, bert_output_path + '/bert_query_by_question_answer_concat.json')

In [28]:
# next, generate the Re-ranked top-k results 
reranked_query_by_question = r.get_reranked_results(bert_query_by_question)
reranked_query_by_answer = r.get_reranked_results(bert_query_by_answer)
reranked_query_by_question_answer = r.get_reranked_results(bert_query_by_question_answer)
reranked_query_by_question_answer_concat = r.get_reranked_results(bert_query_by_question_answer_concat)

2021-02-28 08:53:32 - Re-ranking the top-k results ...
2021-02-28 08:53:32 - Re-ranking the top-k results ...
2021-02-28 08:53:32 - Re-ranking the top-k results ...
2021-02-28 08:53:32 - Re-ranking the top-k results ...


In [29]:
# dump Re-ranked results to json files
reranked_output_path = output_path + "/supervised/" + loss_type + "/" + query_type + "/" + neg_type
make_dirs(reranked_output_path)

dump_to_json(reranked_query_by_question, reranked_output_path + '/reranked_query_by_question.json')
dump_to_json(reranked_query_by_answer, reranked_output_path + '/reranked_query_by_answer.json')
dump_to_json(reranked_query_by_question_answer, reranked_output_path + '/reranked_query_by_question_answer.json')
dump_to_json(reranked_query_by_question_answer_concat, reranked_output_path + '/reranked_query_by_question_answer_concat.json')

**Re-ranking results using query_type="faq"; neg_type="simple"**

In [30]:
# define variables
query_type="faq"; neg_type="simple"; version="1.1"; loss_type='triplet'
bert_model_path='output/FAQIR/models/' + loss_type + '_' + neg_type + '_' + query_type + '_' + version

In [31]:
# create instance of ReRanker class
r = ReRanker(
    bert_model_path=bert_model_path, 
    test_queries=test_queries, relevance_label_df=relevance_label_df, loss_type=loss_type
)

In [32]:
# next, generate BERT top-k preds and save to files
bert_query_by_question = r.get_bert_topk_preds(es_query_by_question)
bert_query_by_answer = r.get_bert_topk_preds(es_query_by_answer)
bert_query_by_question_answer = r.get_bert_topk_preds(es_query_by_question_answer)
bert_query_by_question_answer_concat = r.get_bert_topk_preds(es_query_by_question_answer_concat)

2021-02-28 08:53:33 - Generating BERT top-k results ...
2021-02-28 08:53:33 - Load pretrained SentenceTransformer: output/StackFAQv1.0/models/triplet_simple_faq_1.1
2021-02-28 08:53:33 - Load SentenceTransformer from folder: output/StackFAQv1.0/models/triplet_simple_faq_1.1
2021-02-28 08:53:50 - Use pytorch device: cuda
100%|██████████| 1249/1249 [01:44<00:00, 12.01it/s]
2021-02-28 08:55:34 - Generating BERT top-k results ...
2021-02-28 08:55:34 - Load pretrained SentenceTransformer: output/StackFAQv1.0/models/triplet_simple_faq_1.1
2021-02-28 08:55:34 - Load SentenceTransformer from folder: output/StackFAQv1.0/models/triplet_simple_faq_1.1
2021-02-28 08:55:36 - Use pytorch device: cuda
100%|██████████| 1249/1249 [01:46<00:00, 11.70it/s]
2021-02-28 08:57:22 - Generating BERT top-k results ...
2021-02-28 08:57:22 - Load pretrained SentenceTransformer: output/StackFAQv1.0/models/triplet_simple_faq_1.1
2021-02-28 08:57:22 - Load SentenceTransformer from folder: output/StackFAQv1.0/models/

In [33]:
# dump BERT top-k preds to json files
bert_output_path = output_path + "/supervised/" + loss_type + "/" + query_type + "/" + neg_type
make_dirs(bert_output_path)

dump_to_json(bert_query_by_question, bert_output_path + '/bert_query_by_question.json')
dump_to_json(bert_query_by_answer, bert_output_path + '/bert_query_by_answer.json')
dump_to_json(bert_query_by_question_answer, bert_output_path + '/bert_query_by_question_answer.json')
dump_to_json(bert_query_by_question_answer_concat, bert_output_path + '/bert_query_by_question_answer_concat.json')

In [34]:
# next, generate the Re-ranked top-k results 
reranked_query_by_question = r.get_reranked_results(bert_query_by_question)
reranked_query_by_answer = r.get_reranked_results(bert_query_by_answer)
reranked_query_by_question_answer = r.get_reranked_results(bert_query_by_question_answer)
reranked_query_by_question_answer_concat = r.get_reranked_results(bert_query_by_question_answer_concat)

2021-02-28 09:00:57 - Re-ranking the top-k results ...
2021-02-28 09:00:57 - Re-ranking the top-k results ...
2021-02-28 09:00:57 - Re-ranking the top-k results ...
2021-02-28 09:00:57 - Re-ranking the top-k results ...


In [35]:
# dump Re-ranked results to json files
reranked_output_path = output_path + "/supervised/" + loss_type + "/" + query_type + "/" + neg_type
make_dirs(reranked_output_path)

dump_to_json(reranked_query_by_question, reranked_output_path + '/reranked_query_by_question.json')
dump_to_json(reranked_query_by_answer, reranked_output_path + '/reranked_query_by_answer.json')
dump_to_json(reranked_query_by_question_answer, reranked_output_path + '/reranked_query_by_question_answer.json')
dump_to_json(reranked_query_by_question_answer_concat, reranked_output_path + '/reranked_query_by_question_answer_concat.json')

**##################### Softmax Loss #####################**

**Re-ranking results using query_type="user_query"; neg_type="hard"**

In [36]:
# define variables
query_type="user_query"; neg_type="hard"; version="1.1"; loss_type='softmax'
bert_model_path='output/FAQIR/models/' + loss_type + '_' + neg_type + '_' + query_type + '_' + version

In [37]:
# create instance of ReRanker class
r = ReRanker(
    bert_model_path=bert_model_path, 
    test_queries=test_queries, relevance_label_df=relevance_label_df, loss_type=loss_type
)

In [38]:
# next, generate BERT top-k preds and save to files
bert_query_by_question = r.get_bert_topk_preds(es_query_by_question)
bert_query_by_answer = r.get_bert_topk_preds(es_query_by_answer)
bert_query_by_question_answer = r.get_bert_topk_preds(es_query_by_question_answer)
bert_query_by_question_answer_concat = r.get_bert_topk_preds(es_query_by_question_answer_concat)

2021-02-28 09:00:58 - Generating BERT top-k results ...
2021-02-28 09:01:08 - Use pytorch device: cuda
100%|██████████| 1249/1249 [01:39<00:00, 12.59it/s]
2021-02-28 09:02:47 - Generating BERT top-k results ...
2021-02-28 09:02:49 - Use pytorch device: cuda
100%|██████████| 1249/1249 [01:42<00:00, 12.24it/s]
2021-02-28 09:04:31 - Generating BERT top-k results ...
2021-02-28 09:04:33 - Use pytorch device: cuda
100%|██████████| 1249/1249 [01:41<00:00, 12.33it/s]
2021-02-28 09:06:15 - Generating BERT top-k results ...
2021-02-28 09:06:17 - Use pytorch device: cuda
100%|██████████| 1249/1249 [01:39<00:00, 12.58it/s]


In [39]:
# dump BERT top-k preds to json files
bert_output_path = output_path + "/supervised/" + loss_type + "/" + query_type + "/" + neg_type
make_dirs(bert_output_path)

dump_to_json(bert_query_by_question, bert_output_path + '/bert_query_by_question.json')
dump_to_json(bert_query_by_answer, bert_output_path + '/bert_query_by_answer.json')
dump_to_json(bert_query_by_question_answer, bert_output_path + '/bert_query_by_question_answer.json')
dump_to_json(bert_query_by_question_answer_concat, bert_output_path + '/bert_query_by_question_answer_concat.json')

In [40]:
# next, generate the Re-ranked top-k results 
reranked_query_by_question = r.get_reranked_results(bert_query_by_question)
reranked_query_by_answer = r.get_reranked_results(bert_query_by_answer)
reranked_query_by_question_answer = r.get_reranked_results(bert_query_by_question_answer)
reranked_query_by_question_answer_concat = r.get_reranked_results(bert_query_by_question_answer_concat)

2021-02-28 09:07:57 - Re-ranking the top-k results ...
2021-02-28 09:07:57 - Re-ranking the top-k results ...
2021-02-28 09:07:57 - Re-ranking the top-k results ...
2021-02-28 09:07:57 - Re-ranking the top-k results ...


In [41]:
# dump Re-ranked results to json files
reranked_output_path = output_path + "/supervised/" + loss_type + "/" + query_type + "/" + neg_type
make_dirs(reranked_output_path)

dump_to_json(reranked_query_by_question, reranked_output_path + '/reranked_query_by_question.json')
dump_to_json(reranked_query_by_answer, reranked_output_path + '/reranked_query_by_answer.json')
dump_to_json(reranked_query_by_question_answer, reranked_output_path + '/reranked_query_by_question_answer.json')
dump_to_json(reranked_query_by_question_answer_concat, reranked_output_path + '/reranked_query_by_question_answer_concat.json')

**Re-ranking results using query_type="user_query"; neg_type="simple"**

In [42]:
# define variables
query_type="user_query"; neg_type="simple"; version="1.1"; loss_type='softmax'
bert_model_path='output/FAQIR/models/' + loss_type + '_' + neg_type + '_' + query_type + '_' + version

In [43]:
# create instance of ReRanker class
r = ReRanker(
    bert_model_path=bert_model_path, 
    test_queries=test_queries, relevance_label_df=relevance_label_df, loss_type=loss_type
)

In [44]:
# next, generate BERT top-k preds and save to files
bert_query_by_question = r.get_bert_topk_preds(es_query_by_question)
bert_query_by_answer = r.get_bert_topk_preds(es_query_by_answer)
bert_query_by_question_answer = r.get_bert_topk_preds(es_query_by_question_answer)
bert_query_by_question_answer_concat = r.get_bert_topk_preds(es_query_by_question_answer_concat)

2021-02-28 09:07:58 - Generating BERT top-k results ...
2021-02-28 09:08:09 - Use pytorch device: cuda
100%|██████████| 1249/1249 [01:39<00:00, 12.54it/s]
2021-02-28 09:09:49 - Generating BERT top-k results ...
2021-02-28 09:09:51 - Use pytorch device: cuda
100%|██████████| 1249/1249 [01:42<00:00, 12.20it/s]
2021-02-28 09:11:33 - Generating BERT top-k results ...
2021-02-28 09:11:35 - Use pytorch device: cuda
100%|██████████| 1249/1249 [01:41<00:00, 12.33it/s]
2021-02-28 09:13:16 - Generating BERT top-k results ...
2021-02-28 09:13:18 - Use pytorch device: cuda
100%|██████████| 1249/1249 [01:38<00:00, 12.62it/s]


In [45]:
# dump BERT top-k preds to json files
bert_output_path = output_path + "/supervised/" + loss_type + "/" + query_type + "/" + neg_type
make_dirs(bert_output_path)

dump_to_json(bert_query_by_question, bert_output_path + '/bert_query_by_question.json')
dump_to_json(bert_query_by_answer, bert_output_path + '/bert_query_by_answer.json')
dump_to_json(bert_query_by_question_answer, bert_output_path + '/bert_query_by_question_answer.json')
dump_to_json(bert_query_by_question_answer_concat, bert_output_path + '/bert_query_by_question_answer_concat.json')

In [46]:
# next, generate the Re-ranked top-k results 
reranked_query_by_question = r.get_reranked_results(bert_query_by_question)
reranked_query_by_answer = r.get_reranked_results(bert_query_by_answer)
reranked_query_by_question_answer = r.get_reranked_results(bert_query_by_question_answer)
reranked_query_by_question_answer_concat = r.get_reranked_results(bert_query_by_question_answer_concat)

2021-02-28 09:14:58 - Re-ranking the top-k results ...
2021-02-28 09:14:58 - Re-ranking the top-k results ...
2021-02-28 09:14:58 - Re-ranking the top-k results ...
2021-02-28 09:14:58 - Re-ranking the top-k results ...


In [47]:
# dump Re-ranked results to json files
reranked_output_path = output_path + "/supervised/" + loss_type + "/" + query_type + "/" + neg_type
make_dirs(reranked_output_path)

dump_to_json(reranked_query_by_question, reranked_output_path + '/reranked_query_by_question.json')
dump_to_json(reranked_query_by_answer, reranked_output_path + '/reranked_query_by_answer.json')
dump_to_json(reranked_query_by_question_answer, reranked_output_path + '/reranked_query_by_question_answer.json')
dump_to_json(reranked_query_by_question_answer_concat, reranked_output_path + '/reranked_query_by_question_answer_concat.json')

**Re-ranking results using query_type="faq"; neg_type="hard"**

In [14]:
# define variables
query_type="faq"; neg_type="hard"; version="1.1"; loss_type='softmax'
bert_model_path='output/FAQIR/models/' + loss_type + '_' + neg_type + '_' + query_type + '_' + version

In [15]:
# create instance of ReRanker class
r = ReRanker(
    bert_model_path=bert_model_path, 
    test_queries=test_queries, relevance_label_df=relevance_label_df, loss_type=loss_type
)

In [16]:
# next, generate BERT top-k preds and save to files
bert_query_by_question = r.get_bert_topk_preds(es_query_by_question)
bert_query_by_answer = r.get_bert_topk_preds(es_query_by_answer)
bert_query_by_question_answer = r.get_bert_topk_preds(es_query_by_question_answer)
bert_query_by_question_answer_concat = r.get_bert_topk_preds(es_query_by_question_answer_concat)

2021-02-28 09:30:11 - Generating BERT top-k results ...
2021-02-28 09:30:21 - Use pytorch device: cuda
100%|██████████| 1249/1249 [02:01<00:00, 10.27it/s]
2021-02-28 09:32:23 - Generating BERT top-k results ...
2021-02-28 09:32:25 - Use pytorch device: cuda
100%|██████████| 1249/1249 [01:56<00:00, 10.68it/s]
2021-02-28 09:34:22 - Generating BERT top-k results ...
2021-02-28 09:34:24 - Use pytorch device: cuda
100%|██████████| 1249/1249 [01:56<00:00, 10.74it/s]
2021-02-28 09:36:20 - Generating BERT top-k results ...
2021-02-28 09:36:22 - Use pytorch device: cuda
100%|██████████| 1249/1249 [01:51<00:00, 11.20it/s]


In [17]:
# dump BERT top-k preds to json files
bert_output_path = output_path + "/supervised/" + loss_type + "/" + query_type + "/" + neg_type
make_dirs(bert_output_path)

dump_to_json(bert_query_by_question, bert_output_path + '/bert_query_by_question.json')
dump_to_json(bert_query_by_answer, bert_output_path + '/bert_query_by_answer.json')
dump_to_json(bert_query_by_question_answer, bert_output_path + '/bert_query_by_question_answer.json')
dump_to_json(bert_query_by_question_answer_concat, bert_output_path + '/bert_query_by_question_answer_concat.json')

In [18]:
# next, generate the Re-ranked top-k results 
reranked_query_by_question = r.get_reranked_results(bert_query_by_question)
reranked_query_by_answer = r.get_reranked_results(bert_query_by_answer)
reranked_query_by_question_answer = r.get_reranked_results(bert_query_by_question_answer)
reranked_query_by_question_answer_concat = r.get_reranked_results(bert_query_by_question_answer_concat)

2021-02-28 09:38:15 - Re-ranking the top-k results ...
2021-02-28 09:38:15 - Re-ranking the top-k results ...
2021-02-28 09:38:15 - Re-ranking the top-k results ...
2021-02-28 09:38:15 - Re-ranking the top-k results ...


In [19]:
# dump Re-ranked results to json files
reranked_output_path = output_path + "/supervised/" + loss_type + "/" + query_type + "/" + neg_type
make_dirs(reranked_output_path)

dump_to_json(reranked_query_by_question, reranked_output_path + '/reranked_query_by_question.json')
dump_to_json(reranked_query_by_answer, reranked_output_path + '/reranked_query_by_answer.json')
dump_to_json(reranked_query_by_question_answer, reranked_output_path + '/reranked_query_by_question_answer.json')
dump_to_json(reranked_query_by_question_answer_concat, reranked_output_path + '/reranked_query_by_question_answer_concat.json')

**Re-ranking results using query_type="faq"; neg_type="simple"**

In [20]:
# define variables
query_type="faq"; neg_type="simple"; version="1.1"; loss_type='softmax'
bert_model_path='output/FAQIR/models/' + loss_type + '_' + neg_type + '_' + query_type + '_' + version

In [21]:
# create instance of ReRanker class
r = ReRanker(
    bert_model_path=bert_model_path, 
    test_queries=test_queries, relevance_label_df=relevance_label_df, loss_type=loss_type
)

In [22]:
# next, generate BERT top-k preds and save to files
bert_query_by_question = r.get_bert_topk_preds(es_query_by_question)
bert_query_by_answer = r.get_bert_topk_preds(es_query_by_answer)
bert_query_by_question_answer = r.get_bert_topk_preds(es_query_by_question_answer)
bert_query_by_question_answer_concat = r.get_bert_topk_preds(es_query_by_question_answer_concat)

2021-02-28 09:38:44 - Generating BERT top-k results ...
2021-02-28 09:38:51 - Use pytorch device: cuda
100%|██████████| 1249/1249 [01:53<00:00, 10.98it/s]
2021-02-28 09:40:44 - Generating BERT top-k results ...
2021-02-28 09:40:47 - Use pytorch device: cuda
100%|██████████| 1249/1249 [01:57<00:00, 10.62it/s]
2021-02-28 09:42:44 - Generating BERT top-k results ...
2021-02-28 09:42:46 - Use pytorch device: cuda
100%|██████████| 1249/1249 [01:55<00:00, 10.78it/s]
2021-02-28 09:44:42 - Generating BERT top-k results ...
2021-02-28 09:44:44 - Use pytorch device: cuda
100%|██████████| 1249/1249 [01:51<00:00, 11.21it/s]


In [23]:
# dump BERT top-k preds to json files
bert_output_path = output_path + "/supervised/" + loss_type + "/" + query_type + "/" + neg_type
make_dirs(bert_output_path)

dump_to_json(bert_query_by_question, bert_output_path + '/bert_query_by_question.json')
dump_to_json(bert_query_by_answer, bert_output_path + '/bert_query_by_answer.json')
dump_to_json(bert_query_by_question_answer, bert_output_path + '/bert_query_by_question_answer.json')
dump_to_json(bert_query_by_question_answer_concat, bert_output_path + '/bert_query_by_question_answer_concat.json')

In [24]:
# next, generate the Re-ranked top-k results 
reranked_query_by_question = r.get_reranked_results(bert_query_by_question)
reranked_query_by_answer = r.get_reranked_results(bert_query_by_answer)
reranked_query_by_question_answer = r.get_reranked_results(bert_query_by_question_answer)
reranked_query_by_question_answer_concat = r.get_reranked_results(bert_query_by_question_answer_concat)

2021-02-28 09:46:37 - Re-ranking the top-k results ...
2021-02-28 09:46:37 - Re-ranking the top-k results ...
2021-02-28 09:46:37 - Re-ranking the top-k results ...
2021-02-28 09:46:37 - Re-ranking the top-k results ...


In [25]:
# dump Re-ranked results to json files
reranked_output_path = output_path + "/supervised/" + loss_type + "/" + query_type + "/" + neg_type
make_dirs(reranked_output_path)

dump_to_json(reranked_query_by_question, reranked_output_path + '/reranked_query_by_question.json')
dump_to_json(reranked_query_by_answer, reranked_output_path + '/reranked_query_by_answer.json')
dump_to_json(reranked_query_by_question_answer, reranked_output_path + '/reranked_query_by_question_answer.json')
dump_to_json(reranked_query_by_question_answer_concat, reranked_output_path + '/reranked_query_by_question_answer_concat.json')